In [1]:
import pandas as pd
import os
from os.path import exists
import re
pd.set_option('display.max_columns', None)
path = r'./datasets/Dane_o_przestepczosci_w_latach_19992021/'

In [2]:
# zgwalcenia = pd.read_csv("./datasets/Dane_o_przestepczosci_w_latach_19992021/Zgwacenie/przestepstwa_zgwalcenie.csv",
#                        sep=';', skiprows=2, usecols=[*range(0,4)], encoding = 'cp1250')
# zgwalcenia.drop(zgwalcenia.tail(2).index,inplace=True)
# zgwalcenia['crime']= str(zgwalcenia)
# zgwalcenia.head()

In [ ]:
for (dirpath, dirnames, filenames) in os.walk(path):
    for filename in filenames:
        re1= r'^(.*?) wg'
        re2= '([^\/]+$)'
        cutname = re.compile("(%s|%s)" % (re1, re2)).findall(dirpath)
        # print(dirpath)
        # print("CUT: " + cutname[0][1])
        # oldname = re.findall(re2, dirpath)
        
        if not (exists(cutname[0][1])):
            os.rename(dirpath, cutname[0][1])
        else:
            os.rename(dirpath, cutname[0][1] + " cd")    

In [32]:
crimes = pd.DataFrame()
for (dirpath, dirnames, filenames) in os.walk(path):
    for filename in filenames:
        if re.search("przestepstwa", filename) != None:
            crime = re.findall(r"(?<=_)\w+", filename)
            df_name = pd.read_csv(r"{}".format(dirpath+"/"+filename), sep=';', skiprows=2, usecols=[*range(0,4)], encoding = 'cp1250')
            df_name['Type']= str(crime[0])
            crimes = pd.concat([crimes, df_name], ignore_index=True)
            

crimes = crimes.dropna()
crimes = crimes.rename(columns={'Jednostka podziału administracyjnego':'Area', 'Rok':'Year', 'Przestępstwa stwierdzone':'Crimes established', 'Przestępstwa wykryte':'Crimes detected'})

crimes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6118 entries, 0 to 6160
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Area                6118 non-null   object 
 1   Year                6118 non-null   float64
 2   Crimes established  6118 non-null   object 
 3   Crimes detected     6118 non-null   object 
 4   Type                6118 non-null   object 
dtypes: float64(1), object(4)
memory usage: 286.8+ KB


In [39]:
crimes['Crimes detected'] = crimes['Crimes detected'].str.replace(' ', '')
crimes['Crimes detected'] = pd.to_numeric(crimes['Crimes detected'], errors='coerce')
crimes['Crimes established'] = crimes['Crimes established'].str.replace(' ', '')
crimes['Crimes established'] =  pd.to_numeric(crimes['Crimes established'], errors='coerce')

crimes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6118 entries, 0 to 6160
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Area                6118 non-null   object 
 1   Year                6118 non-null   float64
 2   Crimes established  6080 non-null   float64
 3   Crimes detected     6080 non-null   float64
 4   Type                6118 non-null   object 
dtypes: float64(3), object(2)
memory usage: 286.8+ KB


In [40]:
warsaw_crimes = crimes[crimes['Area']=='teren działania KSP Warszawa']
warsaw_crimes.reset_index(inplace=True, drop=True)
warsaw_crimes.head()

,Area,Year,Crimes established,Crimes detected,Type
0,teren działania KSP Warszawa,2021.0,18642.0,4258.0,kradziez
1,teren działania KSP Warszawa,2020.0,15954.0,3550.0,kradziez
2,teren działania KSP Warszawa,2019.0,18706.0,3976.0,kradziez
3,teren działania KSP Warszawa,2018.0,17386.0,3738.0,kradziez
4,teren działania KSP Warszawa,2017.0,17441.0,3608.0,kradziez


In [44]:
warsaw_crimes.groupby('Type')['Crimes detected'].sum().sort_values()

Type
zabojstwo               1428.0
zgwalcenie              1638.0
kradziez_samochodu      6949.0
korupcja                8206.0
bojka                   9274.0
uszczerbek             13779.0
uszkodzenie            27724.0
włamania               36945.0
kradziez               94196.0
gospodarcze           137368.0
nietrzezwi            140996.0
drogowe               166461.0
17x7                  204234.0
kryminalne            601820.0
Name: Crimes detected, dtype: float64